In [1]:
import os
import cv2
import supervisely as sly
from dotenv import load_dotenv

## Init API client

In [2]:
api = sly.Api.from_env()

In [5]:
workspace_id = int(os.environ["WORKSPACE_ID"])
workspace = api.workspace.get_info_by_id(workspace_id)
if workspace is None:
    print("you should put correct workspaceId value to local.env")
    raise ValueError(f"Workspace with id={workspace_id} not found")

## Create project

In [8]:
project = api.project.create(workspace.id, name="Demo", change_name_if_conflict=True)
dataset = api.dataset.create(project.id, name="berries")
print(f"Project has been sucessfully created, id={project.id}")

Project has been sucessfully created, id=13864


## Create annotation classes

In [9]:
strawberry = sly.ObjClass("strawberry", sly.Rectangle, color=[0, 0, 255])
raspberry = sly.ObjClass("raspberry", sly.Polygon, color=[0, 255, 0])
blackberry = sly.ObjClass("blackberry", sly.Bitmap, color=[255, 255, 0])
berry_center = sly.ObjClass("berry_center", sly.Point, color=[0, 255, 255])
separator = sly.ObjClass("separator", sly.Polyline)  # color will be generated randomly

In [14]:
project_meta = sly.ProjectMeta(
    obj_classes=[strawberry, raspberry, blackberry, berry_center, separator]
)
print(project_meta)

ProjectMeta:
Object Classes
+--------------+-----------+----------------+--------+
|     Name     |   Shape   |     Color      | Hotkey |
+--------------+-----------+----------------+--------+
|  strawberry  | Rectangle |  [0, 0, 255]   |        |
|  raspberry   |  Polygon  |  [0, 255, 0]   |        |
|  blackberry  |   Bitmap  | [255, 255, 0]  |        |
| berry_center |   Point   | [0, 255, 255]  |        |
|  separator   |  Polyline | [138, 15, 105] |        |
+--------------+-----------+----------------+--------+
Tags
+------+------------+-----------------+--------+---------------+--------------------+
| Name | Value type | Possible values | Hotkey | Applicable to | Applicable classes |
+------+------------+-----------------+--------+---------------+--------------------+
+------+------------+-----------------+--------+---------------+--------------------+



In [11]:
api.project.update_meta(project.id, project_meta.to_json())

## Create rectangle

In [15]:
bbox = sly.Rectangle(top=127, left=1726, bottom=1087, right=2560)
label1 = sly.Label(geometry=bbox, obj_class=strawberry)

## Create polygon

In [16]:
polygon = sly.Polygon(
    exterior=[
        [941, 663],
        [976, 874],
        [934, 1096],
        [819, 1196],
        [698, 1228],
        [527, 1081],
        [439, 1090],
        [331, 980],
        [359, 808],
        [452, 698],
        [549, 612],
        [762, 564],
        [879, 605],
    ]
)
label2 = sly.Label(geometry=polygon, obj_class=raspberry)

## Create masks

In [31]:
labels_masks = []
for mask_path in [
    "./data/masks/Blackberry_01.png",
    "./data/masks/Blackberry_02.png",
    "./data/masks/Blackberry_03.png",
]:
    # read only first channel of an image
    image_black_and_white = cv2.imread(mask_path)[:, :, 0]
    
    # supports masks with values (0, 1) or (0, 255) or (False, True)
    mask = sly.Bitmap(image_black_and_white)
    label = sly.Label(geometry=mask, obj_class=blackberry)
    labels_masks.append(label)

## Create image annotation

In [32]:
image_path = "data/berries-01.jpg"
height, width = cv2.imread(image_path).shape[0:2]

# result image annotation
all_labels = [label1, label2]
all_labels.extend(labels_masks)
ann = sly.Annotation(img_size=[height, width], labels=all_labels)

## Upload image with annotation

In [33]:
image_name = sly.fs.get_file_name_with_ext(image_path)
image_info = api.image.upload_path(dataset.id, image_name, image_path)
print(f"Image has been sucessfully uploaded, id={image_info.id}")

Image has been sucessfully uploaded, id=8480921


In [34]:
api.annotation.upload_ann(image_info.id, ann)
print(f"Annotation has been sucessfully uploaded to the image {image_name}")

Annotation has been sucessfully uploaded to the image berries-01.jpg


## Create points

In [36]:
labels_points = []
for [row, col] in [
    [1313, 313],
    [1714, 1061],
    [1318, 1851],
    [554, 1912],
    [190, 808],
    [941, 1094],
]:
    point = sly.Point(row, col)
    label = sly.Label(geometry=point, obj_class=berry_center)
    labels_points.append(label)

## Create polyline

In [37]:
polyline = sly.Polyline(
    [[883, 443], [1360, 803], [1395, 1372], [928, 1676], [458, 1372], [552, 554]]
)
label_line = sly.Label(geometry=polyline, obj_class=separator)

## Upload the second image with annotation 

In [39]:
image_path = "data/berries-02.jpg"
height, width = cv2.imread(image_path).shape[0:2]

In [40]:
# result image annotation
ann = sly.Annotation(img_size=[height, width], labels=[*labels_points, label_line])

# upload image to the dataset on server
image_name = sly.fs.get_file_name_with_ext(image_path)
image_info = api.image.upload_path(dataset.id, image_name, image_path)
print(f"Image has been sucessfully uploaded, id={image_info.id}")

# upload annotation to the image on server
api.annotation.upload_ann(image_info.id, ann)
print(f"Annotation has been sucessfully uploaded to the image {image_name}")

Image has been sucessfully uploaded, id=8480922
Annotation has been sucessfully uploaded to the image berries-02.jpg
